<a href="https://colab.research.google.com/github/NimaZah/Protest/blob/main/Deep_learning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
# imports for notebook boilerplate
!pip install -Uqq fastbook
import fastbook
from fastbook import *
from fastai.tabular.all import *
fastbook.setup_book()

     |████████████████████████████████| 720 kB 32.7 MB/s 
     |████████████████████████████████| 48 kB 6.7 MB/s 
     |████████████████████████████████| 1.2 MB 50.2 MB/s 
     |████████████████████████████████| 189 kB 67.9 MB/s 
     |████████████████████████████████| 55 kB 5.2 MB/s 
     |████████████████████████████████| 558 kB 70.7 MB/s 
     |████████████████████████████████| 51 kB 393 kB/s 
     |████████████████████████████████| 130 kB 20.2 MB/s 
Mounted at /content/gdrive


In [ ]:
path=Path('data/')
path
#downloading data
path.mkdir(parents=True, exist_ok=True)
path.ls()

(#0) []

In [ ]:
# loading the data into a dataframe
df = pd.read_csv('https://raw.githubusercontent.com/joankovacs/mass_mobilization/main/cleaned_protests.csv')
#check the first rows of the dataframe
df.head()
df.drop(['sources', 'notes', 'protesteridentity'], axis=1, inplace=True)
df.head()

,Unnamed: 0,country,year,region,protesterviolence,participants,protest_length,demand_labor_wage_dispute,demand_land_farm_issue,demand_police_brutality,demand_political_behavior_or_process,demand_price_hike_or_tax_policy,demand_removal_of_politician,demand_social_restrictions,y_accomodation,y_ignore,y_adverse_reaction,y_state_violence
0,79,Cuba,2012,Central America,0.0,50.0,1.0,0,0,0,1,0,0,0,0,0,1,0
1,81,Cuba,2014,Central America,0.0,100.0,1.0,1,0,0,0,0,0,0,0,0,1,1
2,82,Cuba,2014,Central America,0.0,50.0,1.0,0,0,0,1,0,0,0,0,0,1,0
3,83,Cuba,2015,Central America,0.0,100.0,1.0,0,0,0,1,0,0,0,0,0,1,0
4,84,Cuba,2015,Central America,0.0,100.0,1.0,0,0,0,1,0,0,0,0,0,1,0


In [ ]:
df.columns

Index(['Unnamed: 0', 'country', 'year', 'region', 'protesterviolence',
       'participants', 'protest_length', 'demand_labor_wage_dispute',
       'demand_land_farm_issue', 'demand_police_brutality',
       'demand_political_behavior_or_process',
       'demand_price_hike_or_tax_policy', 'demand_removal_of_politician',
       'demand_social_restrictions', 'y_accomodation', 'y_ignore',
       'y_adverse_reaction', 'y_state_violence'],
      dtype='object')

In [ ]:
#print the unique values of duration and assign them to unique.
unique = df['protest_length'].unique()
# sort the values of unique.
unique.sort()
unique

In [ ]:
# create a function to assign a category to the values of protest_length column. The function will check if the value 
# of the protest_length is less than or equal to 30, and if it is, then it returns '1 months'. If the value of the
# protest_length column is greater than 30 but less than or equal to 90, then it returns '1-3 months'. If 
# value of the protest_length column is greater than 90 but less than or equal to 180, then it returns
# '1-6 months'. If the value of the protest_length column is greater than 180, then it returns '>6 months'

def protest_length_category(protest_length):
    if protest_length <= 30:
        return '1 month'
    elif protest_length > 30 and protest_length <= 90:
        return '1-3 months'
    elif protest_length > 90 and protest_length <= 180:
        return '3-6 months'
    elif protest_length > 180:
        return '>6 months'

In [ ]:
# Apply the function to the protest_length column.
df['protest_length_category'] = df['protest_length'].apply(protest_length_category)
df.head()

In [ ]:
df['protest_length_category'].unique()

array(['1 month', '>6 months', '1-3 months', '3-6 months'], dtype=object)

In [ ]:
df.drop(['protest_length', 'Unnamed: 0', 'Unnamed: 0'], axis= 1, inplace= True)
df.head()

,country,year,region,protesterviolence,participants,demand_labor_wage_dispute,demand_land_farm_issue,demand_police_brutality,demand_political_behavior_or_process,demand_price_hike_or_tax_policy,demand_removal_of_politician,demand_social_restrictions,y_accomodation,y_ignore,y_adverse_reaction,y_state_violence,protest_length_category
0,Cuba,2012,Central America,0.0,50.0,0,0,0,1,0,0,0,0,0,1,0,1 month
1,Cuba,2014,Central America,0.0,100.0,1,0,0,0,0,0,0,0,0,1,1,1 month
2,Cuba,2014,Central America,0.0,50.0,0,0,0,1,0,0,0,0,0,1,0,1 month
3,Cuba,2015,Central America,0.0,100.0,0,0,0,1,0,0,0,0,0,1,0,1 month
4,Cuba,2015,Central America,0.0,100.0,0,0,0,1,0,0,0,0,0,1,0,1 month


In [ ]:
# export the dataframe to the local machine.
from google.colab import files
df.to_csv('cleaned_protests.csv') 
files.download('cleaned_protests.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# create a directory for the csv file in colab.
!mkdir -p protests
from google.colab import files
uploaded= files.upload()
# move the file to its directory.
!mv cleaned_protests.csv protests/
# associate path with the directory
path = Path('protests')
# ingest the dataset into the df_train DataFrame
df_train=pd.read_csv(path/'cleaned_protests.csv')
df_train.head()

Saving cleaned_protests.csv to cleaned_protests.csv


,Unnamed: 0,country,year,region,protesterviolence,participants,demand_labor_wage_dispute,demand_land_farm_issue,demand_police_brutality,demand_political_behavior_or_process,demand_price_hike_or_tax_policy,demand_removal_of_politician,demand_social_restrictions,y_accomodation,y_ignore,y_adverse_reaction,y_state_violence,protest_length_category
0,0,Cuba,2012,Central America,0.0,50.0,0,0,0,1,0,0,0,0,0,1,0,1 month
1,1,Cuba,2014,Central America,0.0,100.0,1,0,0,0,0,0,0,0,0,1,1,1 month
2,2,Cuba,2014,Central America,0.0,50.0,0,0,0,1,0,0,0,0,0,1,0,1 month
3,3,Cuba,2015,Central America,0.0,100.0,0,0,0,1,0,0,0,0,0,1,0,1 month
4,4,Cuba,2015,Central America,0.0,100.0,0,0,0,1,0,0,0,0,0,1,0,1 month


In [ ]:
# get a count by column of missing values
count = df_train.isna().sum()
df_train_missing = (pd.concat([count.rename('missing_count'),
                     count.div(len(df_train))
                          .rename('missing_ratio')],axis = 1)
             .loc[count.ne(0)])

df_train_missing

,missing_count,missing_ratio


In [ ]:
# define TabularDataLoaders object using the dataframe, the list of pre-processing steps, 
# the categorical and continuous column lists
# procs = [Categorify, FillMissing, Normalize]
# valid_idx = range(len(df_train)-2000, len(df_train))
# dep_var = 'protest_length_category'
# cat_names = ['country', 'region', 'protest_length_category']
# cont_names = ['protest_length']

procs = [FillMissing,Categorify]
dep_var = 'protest_length_category'
cont,cat = cont_cat_split(df_train, 1, dep_var=dep_var) 
print("continuous columns are: ",cont)
print("categorical columns are: ",cat)

continuous columns are:  ['Unnamed: 0', 'year', 'protesterviolence', 'participants', 'demand_labor_wage_dispute', 'demand_land_farm_issue', 'demand_police_brutality', 'demand_political_behavior_or_process', 'demand_price_hike_or_tax_policy', 'demand_removal_of_politician', 'demand_social_restrictions', 'y_accomodation', 'y_ignore', 'y_adverse_reaction', 'y_state_violence']
categorical columns are:  ['country', 'region']


In [ ]:
procs = [FillMissing,Categorify, Normalize]
dls = TabularDataLoaders.from_df(df_train,path,procs= procs, 
                                 cat_names= cat, cont_names = cont, 
                                 y_names = dep_var, 
                                 valid_idx=list(range((df_train.shape[0]-2000),df_train.shape[0])), bs=64)

In [ ]:
# display a sample batch
dls.valid.show_batch()

,country,region,Unnamed: 0,year,protesterviolence,participants,demand_labor_wage_dispute,demand_land_farm_issue,demand_police_brutality,demand_political_behavior_or_process,demand_price_hike_or_tax_policy,demand_removal_of_politician,demand_social_restrictions,y_accomodation,y_ignore,y_adverse_reaction,y_state_violence,protest_length_category
0,Kyrgyzstan,Asia,8110.000021,2011.0,1.000000e+00,1999.999706,1.000000e+00,-4.224892e-10,1.759115e-09,1.288647e-09,3.637797e-09,-6.489525e-10,2.565203e-10,2.750655e-09,1.000000e+00,-2.407500e-09,-3.117339e-09,1-3 months
1,Kyrgyzstan,Asia,8110.999992,2011.0,-8.058123e-09,499.999380,-8.259600e-10,-4.224892e-10,1.759115e-09,1.000000e+00,3.637797e-09,-6.489525e-10,2.565203e-10,2.750655e-09,1.000000e+00,-2.407500e-09,-3.117339e-09,1 month
2,Kyrgyzstan,Asia,8111.999962,2011.0,-8.058123e-09,149.999631,-8.259600e-10,-4.224892e-10,1.759115e-09,1.000000e+00,3.637797e-09,-6.489525e-10,2.565203e-10,2.750655e-09,1.000000e+00,-2.407500e-09,-3.117339e-09,1 month
3,Kyrgyzstan,Asia,8112.999932,2011.0,-8.058123e-09,499.999380,-8.259600e-10,-4.224892e-10,1.759115e-09,1.288647e-09,3.637797e-09,1.000000e+00,2.565203e-10,2.750655e-09,1.000000e+00,-2.407500e-09,-3.117339e-09,1 month
4,Kyrgyzstan,Asia,8113.999903,2011.0,1.000000e+00,5999.999757,-8.259600e-10,-4.224892e-10,1.759115e-09,1.000000e+00,3.637797e-09,1.000000e+00,2.565203e-10,1.000000e+00,-1.098814e-08,-2.407500e-09,-3.117339e-09,1 month
5,Kyrgyzstan,Asia,8114.999873,2011.0,-8.058123e-09,10.000711,-8.259600e-10,-4.224892e-10,1.759115e-09,1.000000e+00,3.637797e-09,-6.489525e-10,2.565203e-10,2.750655e-09,-1.098814e-08,1.000000e+00,-3.117339e-09,1 month
6,Kyrgyzstan,Asia,8116.000123,2011.0,-8.058123e-09,100.000600,-8.259600e-10,-4.224892e-10,1.759115e-09,1.000000e+00,3.637797e-09,-6.489525e-10,2.565203e-10,2.750655e-09,1.000000e+00,-2.407500e-09,-3.117339e-09,1 month
7,Kyrgyzstan,Asia,8117.000093,2011.0,-8.058123e-09,149.999631,-8.259600e-10,-4.224892e-10,1.759115e-09,1.000000e+00,3.637797e-09,-6.489525e-10,2.565203e-10,2.750655e-09,1.000000e+00,-2.407500e-09,-3.117339e-09,1 month
8,Kyrgyzstan,Asia,8118.000063,2011.0,-8.058123e-09,249.999326,-8.259600e-10,-4.224892e-10,1.759115e-09,1.000000e+00,3.637797e-09,1.000000e+00,2.565203e-10,2.750655e-09,1.000000e+00,-2.407500e-09,-3.117339e-09,1 month
9,Kyrgyzstan,Asia,8119.000034,2011.0,-8.058123e-09,200.000295,-8.259600e-10,-4.224892e-10,1.759115e-09,1.000000e+00,3.637797e-09,-6.489525e-10,2.565203e-10,2.750655e-09,1.000000e+00,-2.407500e-09,-3.117339e-09,1 month


In [ ]:
# define and fit the model
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,1.063738,0.662033,0.947500,00:01
1,0.205544,0.112021,0.986000,00:01
2,0.082555,0.092298,0.986000,00:01


In [ ]:
# show the loss function used by the learner
learn.loss_func

FlattenedLoss of CrossEntropyLoss()

In [ ]:
# show a set of results from the model
learn.show_results()

,country,region,Unnamed: 0,year,protesterviolence,participants,demand_labor_wage_dispute,demand_land_farm_issue,demand_police_brutality,demand_political_behavior_or_process,demand_price_hike_or_tax_policy,demand_removal_of_politician,demand_social_restrictions,y_accomodation,y_ignore,y_adverse_reaction,y_state_violence,protest_length_category,protest_length_category_pred
0,111.0,2.0,2.545111,0.638720,-0.562458,-0.145554,-0.425655,-0.200359,-0.267879,0.622885,-0.33379,-0.371169,4.646559,-0.319221,0.880344,-0.74407,-0.344637,0.0,0.0
1,10.0,2.0,2.266189,1.353009,-0.562458,-0.136886,-0.425655,-0.200359,-0.267879,0.622885,-0.33379,-0.371169,-0.215213,3.132623,-1.135919,1.34396,-0.344637,0.0,0.0
2,107.0,2.0,2.119680,-1.682717,1.777990,-0.136886,-0.425655,-0.200359,-0.267879,0.622885,-0.33379,-0.371169,-0.215213,-0.319221,-1.135919,-0.74407,2.901599,0.0,0.0
3,76.0,2.0,1.753621,0.281576,-0.562458,-0.145919,-0.425655,-0.200359,-0.267879,0.622885,-0.33379,-0.371169,-0.215213,-0.319221,0.880344,-0.74407,-0.344637,0.0,0.0
4,107.0,2.0,2.128223,-0.968429,-0.562458,-0.145098,-0.425655,-0.200359,-0.267879,0.622885,-0.33379,-0.371169,-0.215213,-0.319221,-1.135919,1.34396,-0.344637,0.0,0.0
5,97.0,2.0,2.287119,0.638720,-0.562458,-0.143985,-0.425655,4.991044,-0.267879,-1.605433,-0.33379,-0.371169,-0.215213,-0.319221,0.880344,-0.74407,-0.344637,0.0,0.0
6,62.0,2.0,2.073122,-0.432713,1.777990,-0.054769,-0.425655,4.991044,-0.267879,-1.605433,-0.33379,-0.371169,-0.215213,3.132623,-1.135919,-0.74407,-0.344637,0.0,0.0
7,76.0,2.0,1.754476,0.460148,-0.562458,-0.145554,-0.425655,-0.200359,-0.267879,0.622885,-0.33379,-0.371169,-0.215213,-0.319221,0.880344,-0.74407,-0.344637,0.0,0.0
8,128.0,2.0,1.966764,-1.504145,-0.562458,-0.054769,2.349317,-0.200359,-0.267879,-1.605433,-0.33379,-0.371169,-0.215213,-0.319221,0.880344,-0.74407,-0.344637,0.0,0.0


In [ ]:
learn.summary()

In [ ]:
learn.path

Path('protests')

In [ ]:
learn.export('protest.pkl')

In [ ]:
import  pickle
file_path = learn.path/'trained_model.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(learn, f)

In [ ]:
from google.colab import files
files.download(path/'protest.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
learn.path = path
learn.export('trained_model_protests.pkl')
learn.path

Path('protests')

In [ ]:
learn2 = load_learner(path/'trained_model_protests.pkl')

In [ ]:
# get the model predictions
learn2.predict(df_train.iloc[0])

In [ ]:
# get the model predictions for another row
learn2.predict(df_train.iloc[25])

In [ ]:
from google.colab import files
files.download(path/'trained_model_protests.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Step1: Install libraries.
!pip install flask
import json
import os
import urllib.request
import numpy as np
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath
from flask import Flask, render_template, request
#from fastbook import *
from fastai.tabular.all import *

In [ ]:
pathlib.PosixPath= temp
pathlib.PosixPath

pathlib.PosixPath

In [ ]:
!pip install flask-ngrok
%mkdir templates -p
%mkdir static/css -p

In [ ]:
scoring_columns = ['country', 'region']

In [ ]:
# translate the code to Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# create a function to create the model
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# split the data into train and test sets
X = df_train.drop(['protest_length_category'], axis=1)
y = df_train['protest_length_category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# fit the model
model.fit(X_train, y_train)

# evaluate the model
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))